In [1]:
!pip install -q einops peft open_clip_torch protobuf sentencepiece

In [2]:
import sys
import os

sys.path.append(os.getcwd())
import numpy as np
import os
import io
import cv2

import torch

from demo.config import (Config,
                    eval_dict_leaf)

from demo.utils import (retrieve_text,
                  _frame_from_video,
                  setup_internvideo2)

/Users/qing/PycharmProjects/InternVideo/InternVideo2/multi_modality/models/backbones/internvideo2/internvl_clip_vision.py:140: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
/Users/qing/PycharmProjects/InternVideo/InternVideo2/multi_modality/models/backbones/internvideo2/internvideo2.py:135: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)
/Users/qing/PycharmProjects/InternVideo/InternVideo2/multi_modality/models/backbones/internvideo2/internvideo2_clip_vision.py:136: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


No module named 'deepspeed'
deepspeed is not installed!!!


In [3]:
config = Config.from_file('scripts/pretraining/clip/L14/config.py')
config = eval_dict_leaf(config)
config.device = 'cpu'

# Regular sized model
# config.model.text_ckpt_path = 'model/internvl_c_13b_224px.pth'
# config.model.vision_ckpt_path = 'model/InternVideo2.pt'
# config.model.tokenizer_path = 'model/chinese_alpaca_lora_7b'
# config.model.text_encoder.llama_path = 'model/chinese_alpaca_lora_7b'
# config.model.extra_ckpt_path = "model/1B_clip.pth"

# Small Model
config.model.vision_ckpt_path = "model_small/L14_ft_k710_f8.bin"
config.model.text_ckpt_path = "model_small/mobileclip_blt.pt"
config.model.extra_ckpt_path = "model_small/1B_clip.bin"


# model_pth = 'model/InternVideo2.pt'
# config['pretrained_path'] = model_pth
# config['model']['vision_encoder']['pretrained'] = model_pth
# config['model']['extra_ckpt_path'] = 'model/1B_clip.pth'

In [4]:
from models.internvideo2_clip_small import InternVideo2_CLIP_small
intern_model = InternVideo2_CLIP_small(config)

/Users/qing/PycharmProjects/InternVideo/InternVideo2/multi_modality/models/internvideo2_clip_small.py:235: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  vision_ckpt = torch.

In [9]:
from time import perf_counter

video = cv2.VideoCapture('demo/example1.mp4')
frames = [x for x in _frame_from_video(video)]

text_candidates = ["A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon."]

t1 = perf_counter()
texts, probs = retrieve_text(frames, text_candidates, model=intern_model, topk=1, config=config, device = "cpu")

for t, p in zip(texts, probs):
    print(f'text: {t} ~ prob: {p:.4f}')

t2 = perf_counter() - t1
print(t2)

text: A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon. ~ prob: 18.9453
6.012591417000003


In [10]:
import pandas as pd
import numpy as np

val_data = pd.read_csv('k600/val.csv').iloc[:,0:2]
videos = os.listdir('k600/part_0')

print(val_data.head())
print()
print(videos[-5:])

id_label_map = {}
for row in range(len(val_data)):
    id_label_map[val_data.iloc[row]['youtube_id']] = val_data.iloc[row]['label']

remove_points = len(videos) - 500
sampled_points = np.random.choice(videos, remove_points, replace=False)
classes = val_data['label'].unique()
for i in sampled_points:
    os.remove(f'k600/part_0/{i}')
print(f"Removed {remove_points} Videos!")

       label   youtube_id
0  abseiling  0wR5jVB-WPk
1  abseiling  3caPS4FHFF8
2  abseiling  3yaoNwz99xM
3  abseiling  6IbvOJxXnOo
4  abseiling  6_4kjPiQr7w

['-j3eNzQR-EI_000064_000074.mp4', '-C-PvafuvFE_000068_000078.mp4', '0yNXOIqJLtA_000012_000022.mp4', '13Ub1MDkiHc_000014_000024.mp4', '-IlFdaVdEyU_000001_000011.mp4']
Removed 0 Videos!


In [ ]:
top1 = 0
top5 = 0
total = 0
collect_data = []
for check in videos:
    video = cv2.VideoCapture(f'k600/part_0/{check}')
    frames = [x for x in _frame_from_video(video)]
    
    video_label = id_label_map[check[:11]]
    
    texts, probs = retrieve_text(frames, classes, model=intern_model, topk=5, config=config, device = "cpu")
    collect_data.append((texts, probs))
    clear_output(wait=True)
    if texts[0] == video_label:
        top1 += 1
    if video_label in texts:
        top5 += 1
    total += 1
    print(f"{total}/500 Completed")
    print('-'*25)
    print("Correct:", video_label)
    print('-'*25)
    print('Predictions ' + '-'*13)
    for i, v in zip(texts, probs):
        print(f'{v:.4f}', '|', i)
    print('-'*25)
    print("Top 1:",top1/total)
    print("Top 5:", top5/total)